<a href="https://colab.research.google.com/github/williamchi64/williamchi64/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E5%81%9A%E5%9C%96%E5%83%8F%E8%B3%87%E6%96%99%E5%88%86%E5%89%B2%E8%88%87training_inceptionresnet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!wget -P /content/datasets https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
!tar -xf /content/datasets/wiki_crop.tar

--2021-06-09 05:39:34--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 811315200 (774M) [application/x-tar]
Saving to: ‘/content/datasets/wiki_crop.tar’

wiki_crop.tar       100%[===================>] 773.73M  28.4MB/s    in 28s     

2021-06-09 05:40:02 (27.9 MB/s) - ‘/content/datasets/wiki_crop.tar’ saved [811315200/811315200]



In [ ]:
!wget --output-document=/content/datasets/wiki_final.csv drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP&export=download

--2021-06-09 05:40:06--  http://drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP
Resolving drive.google.com (drive.google.com)... 142.251.33.206, 2607:f8b0:4004:837::200e
Connecting to drive.google.com (drive.google.com)|142.251.33.206|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP [following]
--2021-06-09 05:40:06--  https://drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP
Connecting to drive.google.com (drive.google.com)|142.251.33.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cqaq5o0b768chg0i8cnrrrsegmfh041c/1623217200000/01254859074079537426/*/1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP [following]
--2021-06-09 05:40:08--  https://doc-14-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cqaq5o

In [ ]:
# IMDE image 人臉資料集來源: https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os 
import numpy as np
from tqdm import tqdm
import glob
import random

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Dense, Dropout,
                    Activation, Flatten, Conv2D,GlobalAveragePooling2D,
                    MaxPooling2D)
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
wiki = pd.read_csv('/content/datasets/wiki_final.csv')

In [ ]:
wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44183 entries, 0 to 44182
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          44183 non-null  int64  
 1   photo_takens  44183 non-null  int64  
 2   path          44183 non-null  object 
 3   gender        44183 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
def imagepath(index):
  index = index
  # print(wiki.iloc[index,:])
  path = os.path.join('/content/wiki_crop', wiki.iloc[index,2])
  return path

In [ ]:
def imgprocess(size,columns,sample_num):
  imgs = list()
  lbls = list()
  count = len(wiki[columns].unique())
  size = size
  for i in tqdm((sample_num), position=0, leave=True):
    img = cv2.imread(imagepath(i))
    img = cv2.resize(img, size)
    imgs.append(img)
    lbl = int(wiki[columns][i])
    lbl_onehot = np.zeros(count,dtype=float)
    lbl_onehot[lbl] = 1
    lbls.append(lbl_onehot)
  imgs = np.array(imgs)
  lbls = np.array(lbls)
  return {'images':imgs,'labels':lbls}

In [ ]:
# data index for random choosing
random_sample = random.sample(range(0, 44183), 4000)

In [ ]:
data = imgprocess((150,150),'gender',random_sample)

100%|██████████| 4000/4000 [00:06<00:00, 633.01it/s]


In [ ]:
data.keys()

dict_keys(['images', 'labels'])

In [ ]:
data['images'].shape

(4000, 150, 150, 3)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data['images'],data['labels'],test_size=0.2, random_state=500)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2, random_state=500)

In [ ]:
del data

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

(2560, 150, 150, 3)
(2560, 2)
(800, 150, 150, 3)
(800, 2)
(640, 150, 150, 3)
(640, 2)


In [ ]:
num_class = 2
learning_rate = 0.0001
loss = 'categorical_crossentropy'
metrics = 'accuracy'

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2  import InceptionResNetV2,preprocess_input

In [ ]:
model_inresv2 = tf.keras.applications.InceptionResNetV2(
          weights='imagenet', include_top=False,input_shape=(150,150,3)
          )

219062272/219055592 [==============================] - 2s 0us/step


In [ ]:
x = GlobalAveragePooling2D()(model_inresv2.output)
x = Dense(64, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
new_model_inresv2 = Model(inputs=model_inresv2.input, outputs=predictions)

In [ ]:
new_model_inresv2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model_inresv2.trainable=False
new_model_inresv2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model_inresv2.trainable = True

trainable_layer = 3
for layer in model_inresv2.layers[:-trainable_layer]:
    layer.trainable = False

for layer in new_model_inresv2.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f7502993d50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f74fa4b9650> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f74fa43d110> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f74fa4b9ad0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f74f6cd4290> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f74f6cd4d90> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f74f6ceaf10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f74f6cedf90> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f74f6cf0810> False
<tensorflow.python.keras.layers.core.Activation object at 0x7f74f6cf3f90> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f74f6cf7b90> False
<tensorflow.python.keras.layers.c

In [ ]:
optimizer = keras.optimizers.Adam(lr = learning_rate)
new_model_inresv2.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
batch_size = 32
if len(x_train) % batch_size != 0:
  num_steps = len(x_train) // batch_size + 1
else:
  num_steps = len(x_train) // batch_size
num_epochs = 15

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15,
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  horizontal_flip=True,
                  fill_mode='wrap',
                  preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
history = new_model_inresv2.fit_generator(train_datagen.flow(x_train,y_train,batch_size=batch_size),
                        steps_per_epoch=num_steps,
                        epochs=num_epochs,
                        validation_data=test_datagen.flow(x_val,y_val,batch_size=batch_size))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
80/80 [==============================] - 272s 3s/step - loss: 0.3395 - accuracy: 0.8512 - val_loss: 0.3818 - val_accuracy: 0.8844
Epoch 2/15
80/80 [==============================] - 261s 3s/step - loss: 0.2757 - accuracy: 0.8859 - val_loss: 0.2860 - val_accuracy: 0.8984
Epoch 3/15
80/80 [==============================] - 257s 3s/step - loss: 0.2401 - accuracy: 0.9105 - val_loss: 0.2630 - val_accuracy: 0.9078
Epoch 4/15
80/80 [==============================] - 258s 3s/step - loss: 0.2265 - accuracy: 0.9059 - val_loss: 0.2925 - val_accuracy: 0.8875
Epoch 5/15
80/80 [==============================] - 260s 3s/step - loss: 0.2053 - accuracy: 0.9238 - val_loss: 0.2912 - val_accuracy: 0.8813
Epoch 6/15
80/80 [==============================] - 257s 3s/step - loss: 0.1835 - accuracy: 0.9234 - val_loss: 0.2887 - val_accuracy: 0.8938
Epoch 7/15
80/80 [==============================] - 258s 3s/step - loss: 0.1770 - accuracy: 0.9320 - val_loss: 0.2916 - val_accuracy: 0.8734
Epoch 8/15
80

In [ ]:
!mkdir model-logs
new_model_inresv2.save('./model-logs/inceptionresnetV2_best_model.h5')

In [ ]:
test = load_model('./model-logs/inceptionresnetV2_best_model.h5')
valid_generator = test_datagen.flow(x_val,y_val,batch_size=batch_size)
loss,acc = test.evaluate_generator(valid_generator, verbose=2)